In [8]:
### imports
import json
import pandas as pd
from nltk.tokenize import sent_tokenize, TweetTokenizer
from string import punctuation
import re
import regex
tokenizer = TweetTokenizer()
from nltk.corpus import stopwords
stopW = stopwords.words('english')
# stopW = ['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she's", 'should', "should've", 'shouldn', "shouldn't", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 've', 'very', 'was', 'wasn', "wasn't", 'we', 'were', 'weren', "weren't", 'what', 'when', 'where', 'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'won', "won't", 'wouldn', "wouldn't", 'y', 'you', "you'd", "you'll", "you're", "you've", 'your', 'yours', 'yourself', 'yourselves']
# print(sorted(stopW))
import pandas as pd
import math 

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer 

In [10]:
### a method to Read a json file with img captions and article titles and parse it into set of documents

def getDocs(sourcePath):
    medDocs = []
    sourceFile = open(sourcePath, 'r')

    count = 1
    for line in sourceFile:
        medDocs.append(json.loads(line))
        count += 1

    sourceFile.close()
    
    # for k in range(1,count):
    #     print(str(k) + " : " + medDocs[k]['docId'])
    return medDocs


sourcePath = 'assets/medImages.jl'
medDocs = getDocs(sourcePath)
print(medDocs[262]['caption'])
medDocs[262]

Histologic section of adrenal metastatic disease (hematoxylin and eosin, Г—100). Tumor cells arranged in solid nests infiltrating lymph node tissue.


{'articleId': 'PMC7282154',
 'image_path': 'PMC7282154-6_i2376-0605-6-2-e50-f06.jpg',
 'image_url': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7282154/bin/i2376-0605-6-2-e50-f06.jpg',
 'articleUrl': 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7282154/',
 'articleTitle': 'RECURRENT INVASIVE DUCTAL BREAST CARCINOMA PRESENTING AS PRIMARY ADRENAL INSUFFICIENCY WITH ADRENAL CRISIS',
 'caption': 'Histologic section of adrenal metastatic disease (hematoxylin and eosin, Г—100). Tumor cells arranged in solid nests infiltrating lymph node tissue.'}

In [11]:
# method that cleans the title and caption text in each document
# the result is tokenized titles and caption without unwanted brackets, numbers, spaces, tags, unicode charts;
# result includes no punctuation and stopwards and tokens with length less than 2 symbols
# all tokens are lower case


def preprocessContent(content):
    sentences = sent_tokenize(content)
    tokens = []
    for sent in sentences:
        # removes unicode
        cleanedTokens = re.sub(r'[^\x00-\x7F]+', ' ', sent)
        # removes menions
        cleanedTokens = re.sub(r'@\w+', '', cleanedTokens)
        # removes numbers
#         cleanedTokens = re.sub('[0-9]+', '', cleanedTokens)
        # removes nested brackets 
        cleanedTokens = regex.sub(r'\([^()]*+(?:(?R)[^()]*)*+\)', '', cleanedTokens) 
        # removes nested currly brackets
        cleanedTokens = regex.sub('\{(?:[^}{]|\{[^}{]*\})*\}', '', cleanedTokens) 
        # removes html tags
        cleanedTokens = re.sub('\<+/*\w*/*\>+', '', cleanedTokens)
        # removes punctuation
        cleanedTokens = re.sub(r'[%s]' % re.escape(punctuation), ' ', cleanedTokens)
        # removes doubled spaces
        cleanedTokens = re.sub(r'\s{2,}', ' ', cleanedTokens)
        sentTokens = tokenizer.tokenize(cleanedTokens)
        # lower case each token and removes tokens with length less than 2
        sentTokens = [tok.lower() for tok in sentTokens if len(tok) >2]
        # removes stop words 
        sentTokens2 = [tok for tok in sentTokens if not tok.lower() in stopW ]
        
        ### implement stemming! play = played and/or stemming ?!?
        tokens += sentTokens2
    return tokens

from copy import deepcopy
def cleanDocs(medDocs):
    cleanedDocs = deepcopy(medDocs)
#     cleanedDocs = []
    for i, doc in enumerate(cleanedDocs):
        doc['articleTitle'] = preprocessContent(doc['articleTitle'] )
        doc['caption'] = preprocessContent(doc['caption'])
    return cleanedDocs

cleanedDocs = cleanDocs(medDocs)

In [12]:
print(len(cleanedDocs))
# cleanedDocs[262]
# print(cleanedDocs[262]['articleTitle'])
print(medDocs[262]['caption'])
print(cleanedDocs[262]['caption'])

263
Histologic section of adrenal metastatic disease (hematoxylin and eosin, Г—100). Tumor cells arranged in solid nests infiltrating lymph node tissue.
['histologic', 'section', 'adrenal', 'metastatic', 'disease', 'tumor', 'cells', 'arranged', 'solid', 'nests', 'infiltrating', 'lymph', 'node', 'tissue']


In [13]:
def createDocList(corpus):
    titlesList = []
    captionsList = []
    for doc in corpus:
        titlesList.append(doc['articleTitle'])
        captionsList.append(doc['caption'])
    return titlesList, captionsList

titlesList, captionsList = createDocList(medDocs)
print(medDocs[262]['caption'])
print(captionsList[262])
titlesList[262]

Histologic section of adrenal metastatic disease (hematoxylin and eosin, Г—100). Tumor cells arranged in solid nests infiltrating lymph node tissue.
Histologic section of adrenal metastatic disease (hematoxylin and eosin, Г—100). Tumor cells arranged in solid nests infiltrating lymph node tissue.


'RECURRENT INVASIVE DUCTAL BREAST CARCINOMA PRESENTING AS PRIMARY ADRENAL INSUFFICIENCY WITH ADRENAL CRISIS'

In [17]:
cv=CountVectorizer()
title_word_count_vector=cv.fit_transform(titlesList)
title_tokens = cv.get_feature_names()
title_word_count_vector[0]

<1x493 sparse matrix of type '<class 'numpy.int64'>'
	with 13 stored elements in Compressed Sparse Row format>

In [18]:
df = pd.DataFrame(title_word_count_vector, columns=title_tokens)

ValueError: DataFrame constructor not properly called!

In [19]:
caption_word_count_vector=cv.fit_transform(captionsList)

caption_word_count_vector.shape

(263, 3062)

In [20]:
tfidf_transformerTitle=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformerTitle.fit(caption_word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [21]:
tfidf_transformerCaption=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformerCaption.fit(caption_word_count_vector)



TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [24]:
# print idf values 
df_idf = pd.DataFrame(tfidf_transformerTitle.idf_, index=cv.get_feature_names(),columns=["idf_weights"]) 
 
# sort ascending 
# df_idf.sort_values(by=['idf_weights'])

In [23]:
# count matrix 
# count_vector=cv.transform(captionsList) 
titlesVector = cv.transform(titlesList)
captionsVector = cv.transform(captionsList)   
tfidfTransformer = TfidfTransformer(smooth_idf=True)
# tf-idf scores 
tfidfTitle = TfidfTransformer.transform(titlesVector)

TypeError: transform() missing 1 required positional argument: 'X'

In [25]:
feature_names = cv.get_feature_names() 
 
#get tfidf vector for first document 
first_document_vector=tf_idf_vector[262] 
 
#print the scores 
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)

NameError: name 'tf_idf_vector' is not defined